In [6]:
import cv2 as cv
import os
import shutil

In [7]:
def frames_clip(clip_path, frame_rate=5):
    os.makedirs('frames')
    video = cv.VideoCapture(clip_path)
    count = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_path = os.path.join('frames', f"{count:04d}.jpg")
            cv.imwrite(frame_path, frame)
        count += 1
    video.release()

In [8]:
def stitch(frames_folder, clip_axis):
    images = [cv.imread(os.path.join(frames_folder, f)) for f in sorted(os.listdir(frames_folder)) if f.endswith('.jpg')]
    if clip_axis == 'vertical':
     images = [cv.rotate(img, cv.ROTATE_90_CLOCKWISE) for img in images]
    stitcher = cv.Stitcher_create()
    stitcher.setPanoConfidenceThresh(0.7) 
    status, stitched = stitcher.stitch(images)
    if clip_axis == 'vertical':
       stitched = cv.rotate(stitched, cv.ROTATE_90_COUNTERCLOCKWISE)
    if status == cv.Stitcher_OK:
        return stitched
    else:
        print("Stitching failed with error code:", status)
        return None   

In [9]:
def process_video_section(clip_path, output_path, frame_rate=5,axis='vertical'):
    frames_clip(clip_path, frame_rate)
    pano = stitch('frames',clip_axis=axis)
    cv.imwrite(output_path, pano)
    shutil.rmtree('frames')

In [10]:
def process_all_clips(clips_folder, output_folder, frame_rate=5, axis='vertical'):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    clip_files = sorted([f for f in os.listdir(clips_folder) if f.endswith('.mp4')])
    
    for idx, clip_file in enumerate(clip_files):
        clip_path = os.path.join(clips_folder, clip_file)
        output_path = os.path.join(output_folder, f"{idx:04d}.jpg")  # Save outputs in numerical order
        process_video_section(clip_path, output_path, frame_rate=frame_rate, axis=axis)

In [11]:
process_all_clips('clips', 'pano_clips', frame_rate=15, axis='vertical')